# PS3: Optimal Stopping Problem: Computing the Price of an American Call Contract
A binomial lattice model assumes that at each discrete time increment, the state of the system, e.g., the share price of equity, can either increase by a factor $u$ with probability $p$ or decrease by a factor $d$ with probability $(1-p)$ in the next time interval. Thus, each discrete time interval can be modeled as a [Bernoulli random variable](https://en.wikipedia.org/wiki/Bernoulli_distribution):

<div>
    <center>
        <img src="figs/Fig-Binomial-Lattice-Schematic.svg" width="280"/>
    </center>
</div>

while each level (time slice) of the tree is described by a [Binomial distribution](https://en.wikipedia.org/wiki/Binomial_distribution). Different models have been developed to compute the tuple $(u,d,p)$. However, for now, let's specify these and calculate the value of an American `put` contract written with respect to the lattice.

## Contracts
Options are contracts between two parties to buy or sell an asset, e.g., 
shares of stock at a specific price by a specific date:

* A $\textbf{call option}$ gives the owner the right, but not the obligation, to buy the underlying asset at a specific price $K$ by a particular date $T$. The payoff $V_{c}(K, S)$ to the owner (long) of a call contract is given by: $V_{c}(K, S) = \max\left(S-K,0\right)$. The `call` contract seller (short) receives a premium $\mathcal{P}_{c}$ for selling the contract.

* A $\textbf{put option}$ gives the owner the right, but not the obligation, to sell the underlying asset at a specific price $K$ by a particular date $T$. The payoff $V_{p}(K, S)$ to the owner (long) of a put contract is given by: $V_{p}(K, S) = \max\left(K-S,0\right)$. The `put` contract seller (short) receives a premium $\mathcal{P}_{p}$ for selling the contract.

## Learning objectives
The objective of `PS3` is to familiarize students with computing future share prices and the value of American `call` contracts using the Cox, Ross, and Rubinstein (CRR) binomial lattice model. This model uses the same decision rule: `max(exercise, hold)` but has a specific approach to compute the tuple $(u,p,d)$.

Use the [CRR binomial lattice model](https://en.wikipedia.org/wiki/Binomial_options_pricing_model), a particular implementation of the binomial model (see below), to price an American `call` option with the parameters: Initial share price `Sₒ = 60.0` USD/share, a strike price of `K = 60.0` USD/share, a contract duration of `1-year`, implied volatility (IV) equal to `σ = 0.10`, and a risk-free rate of `r̄ = 0.05`.

### Tasks
* __Prerequisite__: Familiarize yourself with the contents of the problem set, and download and compile the required packages (execute the include statement). 
* __Task 1__: Setup and populate the CRR lattice with the parameters provided in the problem.
* __Task 2__: Compute the premium of an American `call` contract computationally (using the `premium(...)` function) for a tree height of `h = 2`
    * `TODO` Test if your calculation is consistent with the `Hull` value using an `@assert isapprox(...)` test.
    * `TODO` confirm the `call` contract premium calculation by hand for `h=2`, scan your work, and include the `pdf` in your repository
* __Task 3__: Repeat __task 2__ with a tree height of `h = 300`, all other parameters held the same.
    * `TODO` Test if your calculation is consistent with the `Hull` value using an `@assert isapprox(...)` test.
    * `Conceptual question`: Why does the increased height improve your estimate of the contract premium?

## Setup

We set up the computational environment by including the `Include.jl` file. The `Include.jl` file loads external packages, various functions that we will use in the exercise, and custom types to model the components of our example problem.

In [1]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/PS3-CHEME-5760-OptionPriceCalculation-Fall-2023/Project.toml`
  No Changes to `~/Desktop/julia_work/PS3-CHEME-5760-OptionPriceCalculation-Fall-2023/Manifest.toml`
Precompiling project...
  ✓ PDMats
  ✓ Distributions
  ✓ Distributions → DistributionsChainRulesCoreExt
  ✓ KernelDensity
  ✓ VLDecisionsPackage
  ✓ StatsPlots
  ✓ VLQuantitativeFinancePackage
  7 dependencies successfully precompiled in 10 seconds. 250 already precompiled.
    Updating git-repo `https://github.com/varnerlab/VLDecisionsPackage.jl.git`
   Resolving package versions...
    Updating `~/Desktop/julia_work/PS3-CHEME-5760-OptionPriceCalculation-Fall-2023/Project.toml`
  [10f378ab] ~ VLDecisionsPackage v0.1.0 `https://github.com/varnerlab/VLDecisionsPackage.jl.git#main` ⇒ v0.1.0 `https://github.com/varnerlab/VLDecisionsPackage.jl.git#main`
    Updating `~/Des

### Types
`Include.jl` loads some [problem-specific types](https://docs.julialang.org/en/v1/manual/types/#Composite-Types) that will be helpful for the lattice model simulation of equity share prices:

The `MyAdjacencyBasedCRREquityPriceTree` encodes the CRR lattice model and has the fields:
    
* The `connectivity::Union{Nothing, Dict{Int64, Array{Int64,1}}}` field holds the indexes of the children for each parent node of the tree.
* The `levels::Union{Nothing, Dict{Int64, Array{Int64,1}}}` field holds indexes of nodes that belong to each level of the tree, i.e., a trading day.
* The `u::Float64` field holds the value of the `up` factor
* The `d::Float64` field holds the value of the `down` factor
* The `p::Float64` field holds the value of the probability of an `up` move.
* The `data::Union{Nothing, Dict{Int64, MyBiomialLatticeEquityNodeModel}}` holds each node in the tree; nodes are of type `MyBiomialLatticeEquityNodeModel`

Each node in the tree is a `MyCRRLatticeNodeModel` type, which has two important fields:
* The `price::Float64` field holds the price value for a node
* The `probability::Float64` field holds the probability value associated with this node
* The `intrinsic::Float64` field holds the `intrinsic` value of the node
* The `extrinsic::Float64` field holds the `extrinsic` value of the node

The `MyAmericanPutContractModel` and `MyAmericanCallContractModel`types hold data associated with American `put` (or `call`) contracts. In this lab, we care about the fields:
* The `K::Float64` field holds the strike price of the contract
* The `DTE::Union{Nothing, Float64}` field holds the duration of the contract (units: years)
* The `sense::Union{Nothing, Int64}` field holds the orientation of the contract (in this lab, `sense = 1`). 

### Functions
`Include.jl` loads the following [Julia functions](https://docs.julialang.org/en/v1/manual/functions/):
  
`function build(model::Type{MyAdjacencyBasedCRREquityPriceTree}, data::NamedTuple) -> MyAdjacencyBasedCRREquityPriceTree` 
> This function takes information in the `data` [NamedTuple](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) argument and returns an instance of the `MyBinomialEquityPriceTree` [mutable type](https://docs.julialang.org/en/v1/manual/types/#Mutable-Composite-Types). Note: the `MyAdjacencyBasedCRREquityPriceTree` returned from the `build(...)` function does not have price or probability information computed yet. Call the `populate(…)` function to populate this data.

`function populate(model::MyAdjacencyBasedCRREquityPriceTree, Sₒ::Float64, h::Int) -> MyBinomialEquityPriceTree`
> The `populate(...)` function takes the `model::MyBinomialEquityPriceTree` instance returned from `build(...)`, a starting share price $S_{o}$ and the height of the tree, i.e., the number of time steps to simulate, and returns an updated `model::MyBinomialEquityPriceTree` instance with the price and probabilities computed for each node in the tree.

`function build(model::Type{MyAmericanPutContractModel}, data::NamedTuple) -> MyAmericanPutContractModel`
> This function takes information in the `data` [NamedTuple](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) argument, the strike price `K`, the duration of the contract `DTE` and the `sense = 1` flag, and returns an instance of a `MyAmericanPutContractModel` model. A similar method is also provided to construct `MyAmericanCallContractModel` instances. 

`function premium(contract::T, model::MyAdjacencyBasedCRREquityPriceTree; choice::Function=_rational) -> Float64 where {T<:AbstractContractModel}`
> The `premium(...)` function takes the `contract::T` and  `model::MyBinomialEquityPriceTree` arguments and returns the `premium::Float64` (price) of the options contract, where the type `T` is any contract type `{call,put}`.

## Task 1: Setup and populate a Cox, Ross, and Rubinstein (CRR) binomial lattice
A binomial lattice model assumes that for each discrete time increment, the state of the system, e.g., the share price of equity, can either increase by a factor $u$ with probability $p$ or decrease by a factor $d$ with probability $(1-p)$. Different models can be developed to compute specific values of the tuple $(u,d,p)$. One particular model that is very widely used in practice is the Cox, Ross, and Rubinstein (CRR) model:

* [Cox, J. C.; Ross, S. A.; Rubinstein, M. (1979). "Option pricing: A simplified approach". Journal of Financial Economics. 7 (3): 229. CiteSeerX 10.1.1.379.7582. doi:10.1016/0304-405X(79)90015-1](https://www.sciencedirect.com/science/article/pii/0304405X79900151?via%3Dihub)

The [CRR binomial lattice model](https://en.wikipedia.org/wiki/Binomial_options_pricing_model) was initially developed for options pricing in 1979. However, one of the critical aspects of estimating an option’s price is calculating the underlying asset’s share price. In the [CRR model](https://en.wikipedia.org/wiki/Binomial_options_pricing_model) model, the `up` and `down` moves are symmetric:

$$ud = 1$$

where the magnitude of an `up` move $u$ is given by:

$$u = \exp(\sigma\sqrt{\Delta{T}})$$

The quantity $\sigma$ denotes a _volatility parameter_, and $\Delta{T}$ represents the time step. The probability $p$ of an `up` move in a [CRR model](https://en.wikipedia.org/wiki/Binomial_options_pricing_model) is given by:

$$p = \frac{\exp(\bar{r}\Delta{T}) - d}{u - d}$$

where $\bar{r}$ denotes a _return parameter_. In the [CRR model](https://en.wikipedia.org/wiki/Binomial_options_pricing_model) model paradigm, the return parameter $\mu$ and the volatility parameter $\sigma$ take on common values:
* The return parameter $\mu$ is a _risk-free_ rate of return; the _risk-free_ rate $\bar{r}$ can be approximated by the [yield on T = 10-year United States Treasury debt security](https://ycharts.com/indicators/10_year_treasury_rate). 
* The volatility parameter $\sigma$ is the [implied volatility](https://www.investopedia.com/terms/i/iv.asp); the implied volatility is the market's view of the likelihood of changes in a given security's price.


### Implementation
To start this calculation, set the problem parameters: the initial share price `Sₒ`, the strike price `K`, the implied volatility `σ̄`, the duration `T` and the time-step `Δt` of the contract, and risk-free rate `r̄`. Initially, let's assume the height of the tree (levels not including the root) is `h = 2`. These parameters are used internally to compute the tuple $(p,u,d)$:

In [2]:
Δt = (1.0/365.0);
T = 365.0*Δt;
Sₒ = 60.0;
K = 60.0;
r̄ = 0.05;
σ̄ = 0.10;

Use the `build(…)` function to create an empty lattice model of type `MyAdjacencyBasedCRREquityPriceTree`. Pass this empty model to the `populate(…)` function using the [Julia piping operator](https://docs.julialang.org/en/v1/manual/functions/#Function-composition-and-piping) `|>`. 

* The `build(...)` for the `MyAdjacencyBasedCRREquityPriceTree` requires the `μ` (risk-free rate), `T` (duration in units of `years`), and `σ` (the implied volatility `IV` value).
* The `populate(…)` requires the initial share price `Sₒ` and tree-height `h` parameters

The `populate(…)` function calculates the prices and probabilities of each node (type `MyCRRLatticeNodeModel`) in the tree. Set the populated lattice to the `lattice_model` variable:

In [3]:
# lattice_model = nothing; # replace this line with your code

In [12]:
lattice_model = VLQuantitativeFinancePackage.build(MyAdjacencyBasedCRREquityPriceTree, (
    μ = r̄, T = T, σ = σ̄)) |> (x-> populate(x, Sₒ = Sₒ, h = 2));

## Task 2: Compute the premium of an American `call` contract with `h = 2`
Now that we have the CRR share price lattice, compute the the premium $\mathcal{P}_{c}$ of an American `call` contract using `backward induction` on the binomial lattice. The `call` contract has a strike price `K = 60` USD/share. The value computed by Hull is $\mathcal{P}_{c} = 4.084$ USD/share. Build `MyAmericanCallContractModel` instance using the `build(...)` method. 

* The `build(...)` method takes the strike price `K` and the `sense=1` parameters

In [5]:
# american_call_contract_model = nothing; # replace this line with your code

In [13]:
american_call_contract_model = VLQuantitativeFinancePackage.build(MyAmericanCallContractModel, (
        K = K, sense = 1));

Call the `premium(...)` pass in the `american_call_contract_model` and `lattice_model` as parameters (in that order). Save the computed premium value in the `my_call_premium` variable:

In [7]:
# my_call_premium = nothing; # replace this line with your code

In [14]:
my_call_premium = premium(american_call_contract_model, lattice_model);

### Check
Use the `@assert isapprox(...)` test, with `rtol = 1e-3` to test if your answer equals (or is close) to the `Hull` value:

In [15]:
hull_call_contract_premium = 4.084;

In [16]:
@assert isapprox(hull_call_contract_premium, my_call_premium, rtol=1e-3)

LoadError: AssertionError: isapprox(hull_call_contract_premium, my_call_premium, rtol = 0.001)

1. `TODO`: Check your `h = 2` calculations by hand (any mistakes?). Scan your hand calculation as `pdf` and include it in the repository
1. `TODO`: Move to __task 3__.

## Task 3: Compute the premium of an American `call` contract with `h = 100`
Repeat __task 2__ with a value of `h = 300` (all other parameters the same). Do the same `@assert isapprox(...)` test, do you pass with a larger tree (why?)

In [11]:
# fill me in